In [1]:
from azureml.core import Workspace, Experiment

#ws = Workspace.get(name="udacity-project")
## Getting available workspace
ws = Workspace.from_config()

ws.write_config(path='.azureml/')

#experiment_name = 'udacity_project_solution_real'

exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code R8UDTGFR7 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-127580
Azure region: southcentralus
Subscription id: 19019985-4709-4996-8830-8568281fc2ff
Resource group: aml-quickstarts-127580


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cluster_name = "cpu-cluster"
'''
try:
    # try to get existing compute
    compute_target = ComputeTarget(workspace=ws, name= cluster_name)
    print("Compute exists")
except ComputeTargetExcpetion:
    print("Compute does not exist. Creating...")
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
'''
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)
compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
compute_target.wait_for_completion(show_output=True, min_node_count = None, timeout_in_minutes=20)
    
#check status of the current cluster
print(compute_target.get_status().serialize())


Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-11-23T17:59:26.183000+00:00', 'errors': None, 'creationTime': '2020-11-23T17:59:21.301898+00:00', 'modifiedTime': '2020-11-23T17:59:37.683748+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [17]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
from azureml.train.hyperdrive.policy import MedianStoppingPolicy
from azureml.train.hyperdrive import uniform, choice

# Specify parameter sampler
#ps = ### YOUR CODE HERE ###

# Logistic Regression needs 2 parameters - 
# C = Regularization Strength (float)
# max_iter = Max iterations (int)
# We use Random Parameter Sampling initially as it supports discrete as well as continuous parameters
# We will try Bayesian Sampling later
# Check initial values for parameters. Current values are used for trial

ps = RandomParameterSampling({
    "C": uniform(1,5),
    "max_iter": choice(range(1, 100))
})

primary_metric_name = "Accuracy"
primary_metric_goal = PrimaryMetricGoal.MAXIMIZE

# Specify early termination policy
#policy = ### YOUR CODE HERE ###
policy = MedianStoppingPolicy(evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
#est = ### YOUR CODE HERE ###
est = SKLearn(source_directory = '.',
              entry_script = 'train.py',
              compute_target = compute_target)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
#hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,
                                     primary_metric_name=primary_metric_name,
                                     primary_metric_goal=primary_metric_goal,
                                     estimator=est,
                                     #max_total_runs=100)
                                     max_total_runs=1)

In [18]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status() == 'Completed')

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_907bdeb1-0fd4-4095-97a5-4c7a17bec99e
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_907bdeb1-0fd4-4095-97a5-4c7a17bec99e?wsid=/subscriptions/19019985-4709-4996-8830-8568281fc2ff/resourcegroups/aml-quickstarts-127580/workspaces/quick-starts-ws-127580

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-11-23T19:23:34.893515][API][INFO]Experiment created<END>\n""<START>[2020-11-23T19:23:35.410151][GENERATOR][INFO]Trying to sample '1' jobs from the hyperparameter space<END>\n""<START>[2020-11-23T19:23:35.583830][GENERATOR][INFO]Successfully sampled '1' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-11-23T19:23:37.1339696Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_907bdeb1-0fd4-4095-97a5-4c7a17bec99e
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_907bdeb1-0fd4-4095-97a5-4c7a17bec99e?wsid=/subscriptions/19019

In [19]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
hyperdrive_best_run = hyperdrive_run.get_best_run_by_primary_metric()
hyperdrive_best_run_mertics = hyperdrive_best_run.get_metrics()
parameter_values = hyperdrive_best_run.get_details()['runDefinition']['arguments']
print('best run id : ', hyperdrive_best_run.id)
print('best run accuracy :', hyperdrive_best_run_mertics['Accuracy'])
print('C : ', parameter_values[1])
print('max_iter : ', parameter_values[3])
print('files : ',hyperdrive_best_run.get_file_names())
# Save best model
best_run_model_name = 'Udacity-hyperdrive-model'
best_run_model_path = './outputs/model.joblib'
#commenting following code

hyperdrive_best_run.download_file('outputs/model.joblib',best_run_model_path)
model = hyperdrive_best_run.register_model(model_name=best_run_model_name,model_path=best_run_model_path)


best run id :  HD_907bdeb1-0fd4-4095-97a5-4c7a17bec99e_0
best run accuracy : 0.9150266691189994
C :  3.290160897759542
max_iter :  61
files :  ['azureml-logs/55_azureml-execution-tvmps_a38af78051fc372c0c5e2680af3b9c639f320a84641b1a86005a8b1c69a67ed5_d.txt', 'azureml-logs/65_job_prep-tvmps_a38af78051fc372c0c5e2680af3b9c639f320a84641b1a86005a8b1c69a67ed5_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_a38af78051fc372c0c5e2680af3b9c639f320a84641b1a86005a8b1c69a67ed5_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/104_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']


In [28]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset
# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = Dataset.Tabular.from_delimited_files(path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")
ds.to_pandas_dataframe()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32945,56,housemaid,married,basic.4y,no,no,yes,cellular,jul,mon,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.960,5228.1,no
32946,37,management,married,university.degree,no,no,yes,cellular,jul,fri,...,7,999,0,nonexistent,1.4,93.918,-42.7,4.957,5228.1,no
32947,26,admin.,single,university.degree,no,no,no,cellular,may,tue,...,4,999,1,failure,-1.8,92.893,-46.2,1.266,5099.1,no
32948,31,blue-collar,single,basic.9y,no,no,no,cellular,apr,mon,...,1,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,no


In [38]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd
# Use the clean_data function to clean your data.
#x, y = clean_data(### YOUR DATA OBJECT HERE ###)
x, y = clean_data(ds)

print("splitting data")
#x_train, y_train, x_test, y_test = train_test_split(x,y,test_size=0.33,random_state=0)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.33,random_state=0)
print('Done splitting')

# Club together training data
df_train = pd.concat([x_train,y_train],axis=1)

splitting data
Done splitting


In [41]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=df_train,
    label_column_name='y',
    n_cross_validations=5)


In [42]:
# Submit your automl run

### YOUR CODE HERE ###
automl_experiment = Experiment(ws,'automl_test_experiment')
automl_run = automl_experiment.submit(config=automl_config)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)
assert(automl_run.get_status() == 'Completed')

WARNING - Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
automl_best_run = automl_run.get_best_run_by_primary_metric()
automl_best_run_mertics = automl_best_run.get_metrics()
parameter_values = automl_best_run.get_details()['runDefinition']['arguments']
print('best run id : ', automl_best_run.id)
print('best run accuracy :', automl_best_run_metrics['Accuracy'])

print('C : ', parameter_values[1])
print('max_iter : ', parameter_values[3])

# Save best model
best_run_model_name = 'Udacity-AutoML-model'
best_run_model_path = './outputs/model.joblib'
#commenting following code

automl_best_run.download_file('outputs/model.joblib',best_run_model_path)
model = automl_best_run.register_model(model_name=best_run_model_name,model_path=best_run_model_path)
